# Irene-run

## Run irene

In [ ]:
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
from __future__ import print_function
import sys
import os
from glob import glob
from time import time

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np
import math

In [ ]:
from invisible_cities.cities.irene import Irene

In [ ]:
from invisible_cities.core.configure import configure, print_configuration

In [ ]:
from invisible_cities.core.system_of_units_c import SystemOfUnits

In [ ]:
units = SystemOfUnits()

In [ ]:
RWF_file = (os.environ['ICDIR']
               + '/database/test_data/irene_bug_Kr_ACTIVE_7bar_RWF.h5')

PMP_file = os.environ['IC_DATA']  + '/irene_bug_Kr_ACTIVE_7bar_PMP.h5'
CONF_file = os.environ['ICDIR'] + '/config/irene.conf'

In [ ]:
def run_irene(rwf_file, pmp_file, conf_file, n_events= 1, store_pmaps=False):
    """Test Irene on a file containing an empty event."""

    CFP = configure(['IRENE',
                             '-c', conf_file,
                             '-i', rwf_file,
                             '-o', pmp_file,
                             '-n', '5'])

    fpp = Irene(run_number=CFP['RUN_NUMBER'])

    files_in = glob(CFP['FILE_IN'])
    files_in.sort()
    fpp.set_input_files(files_in)
    fpp.set_pmap_store(CFP['FILE_OUT'],
                               compression = CFP['COMPRESSION'])
    fpp.set_print(nprint=CFP['NPRINT'],
                  print_empty_events=CFP['PRINT_EMPTY_EVENTS'])

    fpp.set_BLR(n_baseline  = CFP['NBASELINE'],
                        thr_trigger = CFP['THR_TRIGGER'] * units.adc)

    fpp.set_MAU(  n_MAU = CFP['NMAU'],
                        thr_MAU = CFP['THR_MAU'] * units.adc)

    fpp.set_CSUM(thr_csum=CFP['THR_CSUM'] * units.pes)

    fpp.set_S1(tmin   = CFP['S1_TMIN'] * units.mus,
               tmax   = CFP['S1_TMAX'] * units.mus,
               stride = CFP['S1_STRIDE'],
               lmin   = CFP['S1_LMIN'],
               lmax   = CFP['S1_LMAX'])

    fpp.set_S2(tmin   = CFP['S2_TMIN'] * units.mus,
               tmax   = CFP['S2_TMAX'] * units.mus,
               stride = CFP['S2_STRIDE'],
               lmin   = CFP['S2_LMIN'],
               lmax   = CFP['S2_LMAX'])

    fpp.set_SiPM(thr_zs=CFP['THR_ZS'] * units.pes,
                 thr_sipm_s2=CFP['THR_SIPM_S2'] * units.pes)

    t0 = time()
    nevt = fpp.run(nmax=n_events, store_pmaps=store_pmaps)
    t1 = time()
    dt = t1 - t0
    print("IRENE run {} evts in {} s, time/event = {}".\
          format(n_events, dt, dt/n_events))
    

In [ ]:
run_irene(RWF_file, PMP_file, CONF_file, n_events= 1, store_pmaps=False)